In [1]:
import pickle
import numpy as np

import pickle
import random
import pandas as pd
from tqdm import tqdm


from deepctr.feature_column import SparseFeat, VarLenSparseFeat
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

from deepmatch.models import *
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler

from collections import Counter
import collections
import faiss
from deepmatch.utils import recall_N


In [3]:
# 全量
click_df = pd.read_csv('train_click_log.csv')
click_df.columns = click_df.columns.str.strip()
testa_df = pd.read_csv('testA_click_log.csv')
testa_df.columns = testa_df.columns.str.strip()
testb_df = pd.read_csv('testB_click_log.csv')
testb_df.columns = testb_df.columns.str.strip()
article_df = pd.read_csv('articles.csv')
article_df.columns = article_df.columns.str.strip()

merged_df = pd.concat([click_df, testa_df, testb_df], ignore_index=True)
merged_df = pd.merge(merged_df, article_df, left_on='click_article_id', right_on='article_id', how='left')
merged_df = merged_df.drop(columns=['article_id'])
merged_df['click_timestamp'] = merged_df[['click_timestamp']].apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))

test_df = pd.merge(testa_df, article_df, left_on='click_article_id', right_on='article_id', how='left')
test_df['click_timestamp'] = test_df[['click_timestamp']].apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))

In [6]:
# 获取双塔召回时的训练验证数据
# negsample指的是通过滑窗构建样本的时候，负样本的数量
def gen_data_set(data, seq_max_len=10, negsample=0):
    # 用户id，新闻id，正负样本：1 or 0，[历史点击序列]，序列长度，[历史点击序列类目]，新闻id类目，新闻id词数, [历史点击序列词数]
    data.sort_values("click_timestamp", inplace=True)
    item_ids = data['click_article_id'].unique()

    item_id_cateory_map = dict(zip(data['click_article_id'].values, data['category_id'].values))
    wordcount_id_cateory_map = dict(zip(data['click_article_id'].values, data['words_count'].values))
    
    train_set = []
    test_set = []
    
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['click_article_id'].tolist()
        category_list = hist['category_id'].tolist()
        wordcount_list = hist['words_count'].tolist()
        
        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list))   # 用户没看过的文章里面选择负样本
            neg_list = np.random.choice(candidate_set, size=len(pos_list) * negsample, replace=True)  # 对于每个正样本，选择n个负样本
            
        # 长度只有一个的时候，需要把这条数据也放到训练集中，不然的话最终学到的embedding就会有缺失
        # 同时label decode会不成功，因为有些数据就没被encode
        if len(pos_list) == 1:
            hist = pos_list[0]
            category_hist = category_list[0]
            wordcount_hist = wordcount_list[0]
            
            train_set.append((reviewerID, 
                                pos_list[0], 
                                1, 
                                [hist], 
                                seq_len, 
                                [category_hist],
                                category_list[0],
                                wordcount_list[0],
                                [wordcount_hist])) 

        # 全量从0开始
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            category_hist = category_list[:i]
            wordcount_hist = wordcount_list[:i]
            seq_len = min(i, seq_max_len)
            
            if i != len(pos_list) - 1:
                train_set.append((
                    reviewerID, 
                    pos_list[i], 
                    1, 
                    hist[::-1][:seq_len], 
                    seq_len, 
                    category_hist[::-1][:seq_len],
                    category_list[i],
                    wordcount_list[i],
                    wordcount_hist[::-1][:seq_len]))
                
                # 负样本
                for negi in range(negsample):
                    train_set.append((reviewerID, 
                                      neg_list[i * negsample + negi], 
                                      0, 
                                      hist[::-1][:seq_len], 
                                      seq_len,
                                      category_hist[::-1][:seq_len], 
                                      item_id_cateory_map[neg_list[i * negsample + negi]], 
                                      wordcount_id_cateory_map[neg_list[i * negsample + negi]],
                                      wordcount_hist[::-1][:seq_len]))
            
            else:
                # 测试集
                test_set.append((
                                 reviewerID, 
                                 pos_list[i], 
                                 1, 
                                 hist[::-1][:seq_len], 
                                 seq_len, 
                                 category_hist[::-1][:seq_len],
                                 category_list[i],
                                 wordcount_list[i],
                                 wordcount_hist[::-1][:seq_len]))       
            
            
    random.shuffle(train_set)
    random.shuffle(test_set)
    
    return train_set, test_set
    # 用户id，新闻id，正负样本：1 or 0，[历史点击序列]，序列长度，[历史点击序列类目]，新闻id类目，新闻id词数, [历史点击序列词数]


# 将输入的数据进行padding，使得序列特征的长度都一致
# userprofile = ["user_id", 'click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type']
# itemprofile = ["category_id", "words_count"]
def gen_model_input(train_set, user_profile, seq_max_len):
    # 0       1       2                 3               4         5                   6           7
    # 用户id，新闻id，正负样本：1 or 0，[历史点击序列]，序列长度，[历史点击序列类目]，新闻id类目，新闻id词数
    train_uid = np.array([line[0] for line in train_set])
    train_iid = np.array([line[1] for line in train_set])
    train_label = np.array([line[2] for line in train_set])
    train_seq = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])
    train_category_seq = np.array([line[5] for line in train_set])
    train_category = np.array([line[6] for line in train_set])
    train_wordcount = np.array([line[7] for line in train_set])
    train_wordcount_seq = np.array([line[8] for line in train_set])
    

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_seq_category_pad = pad_sequences(train_category_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_seq_wordcount_pad = pad_sequences(train_wordcount_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    
    train_model_input = {'user_id': train_uid,
                         'click_article_id': train_iid,
                         'hist_click_article_id': train_seq_pad,
                         'hist_category_id': train_seq_category_pad,
                         'hist_len': train_hist_len,
                         'category': train_category,
                         'wordcount': train_wordcount,
                         'hist_words_count': train_seq_wordcount_pad}
    
    for key in ['click_environment', 'click_deviceGroup', 'click_os', 'click_country', 'click_region', 'click_referrer_type']:
        train_model_input[key] = user_profile.loc[train_model_input['user_id']][key].values

    return train_model_input, train_label

In [7]:
data = merged_df.copy()

sparse_features = ["user_id", "click_article_id",
                    'click_environment', 'click_deviceGroup', 
                    'click_os', 'click_country', 'click_region', 
                    'click_referrer_type', 'category_id',
                    'words_count']
SEQ_LEN = 10 # 用户点击序列的长度，短的填充，长的截断
negsample = 0

# 类别编码
# NOTE：这个label_encoders = {}是为了记录每一个特征的label encoder
# 到时候解码的时候可以用对应的特征编码器来解码。
label_encoders = {}
feature_max_idx = {}
for feature in sparse_features:
    lbe = LabelEncoder()
    # 如果包含了只有len为1的数据的时候，这里就不要+1了
    data[feature] = lbe.fit_transform(data[feature])
    feature_max_idx[feature] = data[feature].max() + 1
    # 逆转编码
    label_encoders[feature] = lbe # 保存对每一个特征的编码器
    
    
user_profile = data[["user_id", 'click_environment', 
                    'click_deviceGroup', 'click_os', 
                    'click_country', 'click_region', 
                    'click_referrer_type']].drop_duplicates('user_id')

item_profile = data[["click_article_id",
                     'category_id',
                     'words_count']].drop_duplicates('click_article_id')

user_profile.set_index("user_id", inplace=True, drop=False)

user_item_list = data.groupby("user_id")['click_article_id'].apply(list)

embedding_dim = 16

user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], 16),
                        SparseFeat('click_environment', feature_max_idx['click_environment'], 16),
                        SparseFeat('click_deviceGroup', feature_max_idx['click_deviceGroup'], 16),
                        SparseFeat('click_country', feature_max_idx['click_country'], 16),
                        SparseFeat('click_region', feature_max_idx['click_region'], 16),
                        SparseFeat('click_referrer_type', feature_max_idx['click_referrer_type'], 16),
                        VarLenSparseFeat(SparseFeat('hist_click_article_id', feature_max_idx['click_article_id'], embedding_dim,
                                                     embedding_name='article_id'), SEQ_LEN, 'mean', 'hist_len'),
                        VarLenSparseFeat(SparseFeat('hist_category_id', feature_max_idx['category_id'], embedding_dim,
                                                     embedding_name='category_id'), SEQ_LEN, 'mean', 'hist_len'),
                        VarLenSparseFeat(SparseFeat('hist_words_count', feature_max_idx['words_count'], embedding_dim,
                                embedding_name='words_count'), SEQ_LEN, 'mean', 'hist_len')
                        ]

item_feature_columns = [SparseFeat('click_article_id', feature_max_idx['click_article_id'], embedding_dim),]


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
train_set, test_set = gen_data_set(data, SEQ_LEN, negsample)

train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN)

100%|██████████| 300000/300000 [00:49<00:00, 6107.21it/s]


In [10]:
train_counter = Counter(train_model_input['click_article_id'])
item_count = [train_counter.get(i, 0) for i in range(item_feature_columns[0].vocabulary_size)]
sampler_config_youtubednn = NegativeSampler('frequency', num_sampled=255, item_name="click_article_id", item_count=item_count)
# model = YoutubeDNN(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(128,64, embedding_dim), sampler_config=sampler_config)
comirec = ComiRec(user_feature_columns, item_feature_columns, k_max=2, user_dnn_hidden_units=(64, embedding_dim), sampler_config=sampler_config_youtubednn)

comirec.compile(optimizer="adam", loss=sampledsoftmaxloss)

history = comirec.fit(train_model_input, train_label,  # train_label,
                    batch_size=512, epochs=5, verbose=1, validation_split=0.0, )

Train on 1841158 samples
Epoch 1/5
1841158/1841158 [==============================] - 107s 58us/sample - loss: 4.6601
Epoch 2/5
1841158/1841158 [==============================] - 95s 51us/sample - loss: 3.8881
Epoch 3/5
1841158/1841158 [==============================] - 86s 47us/sample - loss: 3.5834
Epoch 4/5
1841158/1841158 [==============================] - 94s 51us/sample - loss: 3.3813
Epoch 5/5
1841158/1841158 [==============================] - 103s 56us/sample - loss: 3.2284


In [9]:
# 记得类别编码
testa_df = pd.read_csv('testA_click_log.csv')
testa_df.columns = testa_df.columns.str.strip()

article_df = pd.read_csv('articles.csv')
article_df.columns = article_df.columns.str.strip()

testa_df = pd.merge(testa_df, article_df, left_on='click_article_id', right_on='article_id', how='left')
testa_df['click_timestamp'] = testa_df[['click_timestamp']].apply(lambda x : (x-np.min(x))/(np.max(x)-np.min(x)))


# NOTE: 这里想要做一个全量数据的，就是把最后一个点击也包括进去的那种
def gen_train_set_no_shuffle(data, seq_max_len=10, negsample=0):
    # 用户id，新闻id，正负样本：1 or 0，[历史点击序列]，序列长度，[历史点击序列类目]，新闻id类目，新闻id词数, [历史点击序列词数]
    data.sort_values("click_timestamp", inplace=True)
    item_ids = data['click_article_id'].unique()

    item_id_cateory_map = dict(zip(data['click_article_id'].values, data['category_id'].values))
    wordcount_id_cateory_map = dict(zip(data['click_article_id'].values, data['words_count'].values))
    
    train_set = []
    
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['click_article_id'].tolist()
        category_list = hist['category_id'].tolist()
        wordcount_list = hist['words_count'].tolist()
        
        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list))   # 用户没看过的文章里面选择负样本
            neg_list = np.random.choice(candidate_set, size=len(pos_list) * negsample, replace=True)  # 对于每个正样本，选择n个负样本
            
        # 长度只有一个的时候，需要把这条数据也放到训练集中，不然的话最终学到的embedding就会有缺失
        # 同时label decode会不成功，因为有些数据就没被encode
        if len(pos_list) == 1:
            hist = pos_list[0]
            category_hist = category_list[0]
            wordcount_hist = wordcount_list[0]
            
            train_set.append((reviewerID, 
                                pos_list[0], 
                                1, 
                                [hist], 
                                seq_len, 
                                [category_hist],
                                category_list[0],
                                wordcount_list[0],
                                [wordcount_hist])) 

        # 全量从0开始
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]
            category_hist = category_list[:i]
            wordcount_hist = wordcount_list[:i]
            seq_len = min(i, seq_max_len)
            
            if i != len(pos_list) - 1:
                train_set.append((
                    reviewerID, 
                    pos_list[i], 
                    1, 
                    hist[::-1][:seq_len], 
                    seq_len, 
                    category_hist[::-1][:seq_len],
                    category_list[i],
                    wordcount_list[i],
                    wordcount_hist[::-1][:seq_len]))
                
                # 负样本
                for negi in range(negsample):
                    train_set.append((reviewerID, 
                                      neg_list[i * negsample + negi], 
                                      0, 
                                      hist[::-1][:seq_len], 
                                      seq_len,
                                      category_hist[::-1][:seq_len], 
                                      item_id_cateory_map[neg_list[i * negsample + negi]], 
                                      wordcount_id_cateory_map[neg_list[i * negsample + negi]],
                                      wordcount_hist[::-1][:seq_len]))
            else:
                # 测试集
                train_set.append((
                                 reviewerID, 
                                 pos_list[i], 
                                 1, 
                                 hist[::-1][:seq_len], 
                                 seq_len, 
                                 category_hist[::-1][:seq_len],
                                 category_list[i],
                                 wordcount_list[i],
                                 wordcount_hist[::-1][:seq_len]))
            
            
    random.shuffle(train_set)
    
    return train_set



# 用来预测新的测试集
# NOTE: 这个出来的历史点击序列包含所有历史点击，而不是之前的把最后一个拿出来
def gen_pred_set(data, seq_max_len=10):
    # 用户id，新闻id，正负样本：1 or 0，[历史点击序列]，序列长度，[历史点击序列类目]，新闻id类目，新闻id词数, [历史点击序列词数]
    data.sort_values("click_timestamp", inplace=True)

    test_set = []
    
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['click_article_id'].tolist()
        category_list = hist['category_id'].tolist()
        wordcount_list = hist['words_count'].tolist()
        
        hist = pos_list[:len(pos_list)]
        category_hist = category_list[:len(pos_list)]
        wordcount_hist = wordcount_list[:len(pos_list)]
        seq_len = min(len(pos_list), seq_max_len)
        
        test_set.append((
                    reviewerID, 
                    pos_list[-1], 
                    1, 
                    hist[::-1][:seq_len], 
                    seq_len, 
                    category_hist[::-1][:seq_len],
                    category_list[-1],
                    wordcount_list[-1],
                    wordcount_hist[::-1][:seq_len]))   
          
    random.shuffle(test_set)
    
    return test_set


# NOTE: 这个出来的是n - 1个历史点击，新闻id是最后一个点击的
def gen_test_pred_set(data, seq_max_len=10):
    # 用户id，新闻id，正负样本：1 or 0，[历史点击序列]，序列长度，[历史点击序列类目]，新闻id类目，新闻id词数, [历史点击序列词数]
    data.sort_values("click_timestamp", inplace=True)

    test_set = []
    
    for reviewerID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['click_article_id'].tolist()
        category_list = hist['category_id'].tolist()
        wordcount_list = hist['words_count'].tolist()
        
        hist = pos_list[:len(pos_list) - 1]
        category_hist = category_list[:len(pos_list) - 1]
        wordcount_hist = wordcount_list[:len(pos_list) - 1]
        seq_len = min(len(pos_list), seq_max_len)
        
        test_set.append((
                    reviewerID, 
                    pos_list[-1], 
                    1, 
                    hist[::-1][:seq_len], 
                    seq_len, 
                    category_hist[::-1][:seq_len],
                    category_list[-1],
                    wordcount_list[-1],
                    wordcount_hist[::-1][:seq_len]))   
          
    random.shuffle(test_set)
    
    return test_set


# NOTE：用testA来预测
testa_df = pd.read_csv('testA_click_log.csv')
testa_df.columns = testa_df.columns.str.strip()

testa_df = pd.merge(testa_df, article_df, left_on='click_article_id', right_on='article_id', how='left')
testa_df = testa_df.drop(columns=['article_id'])

# 测试集编码
for feature in sparse_features:
    if feature in label_encoders:
        lbe = label_encoders[feature]
        testa_df[feature] = lbe.transform(testa_df[feature])


In [13]:
test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN)

In [14]:
# 4. Generate user features for testing and full item features for retrieval
test_user_model_input = test_model_input
all_item_model_input = {"click_article_id": item_profile['click_article_id'].values,}

user_embedding_model = Model(inputs=comirec.user_input, outputs=comirec.user_embedding)
item_embedding_model = Model(inputs=comirec.item_input, outputs=comirec.item_embedding)

user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
# user_embs = user_embs[:, i, :]  # i in [0,k_max) if MIND
item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

print(user_embs.shape)
print(item_embs.shape)

(283066, 2, 16)
(41047, 16)


In [15]:
import heapq
from collections import defaultdict

k_max = 2
topN = 5
test_true_label = {line[0]: [line[1]] for line in test_set}

index = faiss.IndexFlatIP(embedding_dim)
# faiss.normalize_L2(item_embs)
index.add(item_embs)
# faiss.normalize_L2(user_embs)

retrieval_items = defaultdict(list)

if len(user_embs.shape) == 2:  # multi interests model's shape = 3 (MIND,ComiRec)
    user_embs = np.expand_dims(user_embs, axis=1)

score_dict = defaultdict(dict)
for k in range(k_max):
    user_emb = user_embs[:, k, :]
    D, I = index.search(np.ascontiguousarray(user_emb), topN)
    for i, uid in tqdm(enumerate(test_user_model_input['user_id']), total=len(test_user_model_input['user_id'])):
        if np.abs(user_emb[i]).max() < 1e-8:
            continue
        for score, itemid in zip(D[i], I[i]):
            score_dict[uid][itemid] = max(score, score_dict[uid].get(itemid, float("-inf")))

s = []
hit = 0
for i, uid in enumerate(test_user_model_input['user_id']):
    pred = [item_profile['click_article_id'].values[x[0]] for x in
            heapq.nlargest(topN, score_dict[uid].items(), key=lambda x: x[1])]
    filter_item = None
    recall_score = recall_N(test_true_label[uid], pred, N=topN)
    s.append(recall_score)
    if test_true_label[uid] in pred:
        hit += 1
    
    retrieval_items[uid] = pred

print("recall", np.mean(s))
print("hr", hit / len(test_user_model_input['user_id']))

100%|██████████| 283066/283066 [00:05<00:00, 51600.29it/s]


recall 0.24987105480700614
hr 0.24987105480700614


In [16]:
# 把召回出来的dic，转换成df （label encoded）
retrieval_items_df = pd.DataFrame.from_dict(retrieval_items, orient='index', columns=['article_1', 'article_2', 'article_3', 'article_4', 'article_5'])

# 重置索引并将索引设置为user_id
retrieval_items_df.reset_index(inplace=True)
retrieval_items_df.rename(columns={'index': 'user_id'}, inplace=True)

# 测试用户最后一次点击的文章列表（label encoded）
test_true_label_df = pd.DataFrame([(user_id, article_id[0]) for user_id, article_id in test_true_label.items()], columns=['user_id', 'click_article_id'])

# 为了计算MRR和HR，将召回文章列表和真实点击的文章列表merge到一起
merged_retrieval_true_label_df = pd.merge(retrieval_items_df, test_true_label_df, on='user_id')

# 用记录下来的每一个特征的label encoder，来decode数据
id_encoder = label_encoders['user_id']
merged_retrieval_true_label_df['user_id'] = id_encoder.inverse_transform(merged_retrieval_true_label_df['user_id'])

# 愚蠢但有用
article_id_encoder = label_encoders['click_article_id']
merged_retrieval_true_label_df['article_1'] = article_id_encoder.inverse_transform(merged_retrieval_true_label_df['article_1'])
merged_retrieval_true_label_df['article_2'] = article_id_encoder.inverse_transform(merged_retrieval_true_label_df['article_2'])
merged_retrieval_true_label_df['article_3'] = article_id_encoder.inverse_transform(merged_retrieval_true_label_df['article_3'])
merged_retrieval_true_label_df['article_4'] = article_id_encoder.inverse_transform(merged_retrieval_true_label_df['article_4'])
merged_retrieval_true_label_df['article_5'] = article_id_encoder.inverse_transform(merged_retrieval_true_label_df['article_5'])
merged_retrieval_true_label_df['click_article_id'] = article_id_encoder.inverse_transform(merged_retrieval_true_label_df['click_article_id'])

In [17]:
# 计算MRR
def reciprocal_rank(row):
    recommended_articles = [row[f'article_{i}'] for i in range(1, 6)]
    try:
        rank = recommended_articles.index(row['click_article_id']) + 1
        return 1 / rank
    except ValueError:
        return 0

merged_retrieval_true_label_df['reciprocal_rank'] = merged_retrieval_true_label_df.apply(reciprocal_rank, axis=1)

# 计算MRR
mrr = merged_retrieval_true_label_df['reciprocal_rank'].mean()
print(f'MRR: {mrr}')

MRR: 0.1400512012510611


In [18]:
# 计算hit rate
def hit_rate(row):
    recommended_articles = [row[f'article_{i}'] for i in range(1, 6)]
    return int(row['click_article_id'] in recommended_articles)

merged_retrieval_true_label_df['hit'] = merged_retrieval_true_label_df.apply(hit_rate, axis=1)

# 计算Hit Rate
hit_rate_value = merged_retrieval_true_label_df['hit'].mean()
print(f'Hit Rate: {hit_rate_value}')

Hit Rate: 0.24987105480700614


In [19]:
result_df = merged_retrieval_true_label_df[['user_id', 'article_1', 'article_2', 'article_3', 'article_4', 'article_5']]
result_df.sort_values(by='user_id', inplace=True)
result_df

/home/ubuntu-22-04-03/anaconda3/envs/recsys/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,user_id,article_1,article_2,article_3,article_4,article_5
145702,0,331116,277107,177442,336254,235105
209477,1,331116,289003,336254,157478,199372
137270,2,50644,36162,70986,331116,277107
170271,3,50644,36162,70986,209122,206415
201567,4,42757,69129,50494,159019,73506
...,...,...,...,...,...,...
117883,299994,160974,161178,50504,157077,273616
196634,299995,336220,158794,289090,202388,233688
36688,299997,284410,160131,168623,323550,286321
216353,299998,336223,119193,271261,48403,59057


In [24]:
result_df.to_csv('ans_comirec.csv', index=False)